In [1]:
import requests
import os
import asyncio
import requests
import matplotlib.pyplot as plt
from datetime import datetime
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

In [2]:
def get_food_info(product_name):
    url = f"https://world.openfoodfacts.org/cgi/search.pl?action=process&search_terms={product_name}&json=true"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        products = data.get('products', [])
        if products:  # Проверяем, есть ли найденные продукты
            first_product = products[0]
            return {
                'name': first_product.get('product_name', 'Неизвестно'),
                'calories': first_product.get('nutriments', {}).get('energy-kcal_100g', 0)
            }
        return None
    print(f"Ошибка: {response.status_code}")
    return None

In [3]:
get_food_info('banana')

{'name': 'Yogurt Bnine BANANA', 'calories': 88.1}

In [4]:
class FoodInfo(BaseModel):
    name: str = Field(description="Название продукта")
    calories: float = Field(description="Калорийность на 100г")

In [ ]:
async def get_food_info_llm(product_name):
    llm = ChatOpenAI(
        openai_api_key=os.getenv("LLM_API_KEY"),
        base_url=os.getenv("LLM_BASE_URL"),
        model_name=os.getenv("LLM_MODEL_NAME")
    )
    parser = PydanticOutputParser(pydantic_object=FoodInfo)
    prompt = PromptTemplate(
        template="Оцени калорийность продукта: {product}.\n{format_instructions}",
        input_variables=["product", "format_instructions"]
    )
    partial_prompt = prompt.partial(format_instructions=parser.get_format_instructions())
    chain = partial_prompt | llm | parser
    try:
        result = await chain.ainvoke({"product": product_name})
        return {
            'name': result.name,
            'calories': result.calories
        }
    except Exception as e:
        print(f"Ошибка АААААААА УЖАС: {e}")
        return None

In [6]:
food_data = await get_food_info_llm('apple')
food_data

{'name': 'apple', 'calories': 52.0}